# SHIP: network building

This tutorial will cover the essentials for building a network object in SHIP.

## Instate a network object
We here illustrate how to generate and simulate a network. We first import the necessary class, and instate an empty network "net", using the **network** class:

In [35]:
import sys
sys.path.append('C:\\') # to be edited by the user

from SHIP import network
net = network('optional_id') # <- the network __init__ method can take an optional value to be used as a unique identifier.

## Add groups
After the object instantiation, we can start adding our groups of components.
We initially see an example that uses a Poisson spike generator as the driving force for the network temporal evolution. We import the corresponding class, here **poissonN**. We then add it as a group to our network, using the ```network.add``` method:

In [36]:
from SHIP import poissonN
net.add(poissonN, 'P')

The ```network.add``` method requires two mandatory positional arguments: the group *class*, and the group *tag*. In the example above, we provided the class **poissonN** as the first argument, and the name 'P' as the group *tag*. We underline that each tag must be unique, as those will be essential to handle each unique group within the network.

### Set group parameters
The group models here provided come with pre-defined parameters. However, it is of course highly important to tune the parameters of each group or component according to the desire of the user.

At this stage of the development, there is no built-in method that evidences which variables, functions and parameters can be provided during the building stage. We assume that the user can read the model source to rapidly determine those.

The most important parameter is the number of components of the group, *N*, which must be provided in order for SHIP to know how many units to initialize. Another argument is the firing rate *rate*, at which each component spikes.

A way forward to set these parameters is to pass the correlated keyword arguments to the ```network.add``` method. However, The same operation can be carried out in a second moment, by using the ```network.set_params``` method. We give an example below:

In [37]:
net.set_params('P',N=5,rate=100)

With this instruction, we set the 'P' so to contains 20 units, each firing at the fixed rate of 100 Hz.
Let's add a second neuron group, named 'O', using the leaky integrate neuron model **liN**. We pass as keyword arguments the number of units *N* = 2, and the temporal constant *tau_beta* = 20ms:

In [38]:
from SHIP import liN
net.add(liN, 'O', N=2, tau_beta = 20e-3)

Here we defined a group and all its necessary arguments in a single line.
We underline that any argument passed to the network, or internal state, can be retrieved at any time through the property ```groups``` of the **network** object. ```network.groups``` is a dict data structure, where each key give access to the properties of each group. 

The dict is here modified so to allow using a more convenient dot notation.  Thus, any accessible variable can be retrieved using a nested dot notation.  The bespoke datatype is called Dict (with the uppercase D).

For instance, we would be able to retrieve the value of ```tau_beta``` of 'O' as follows (after initializing the network):

In [39]:
net.groups.O.tau_beta
# alternatively: net.groups['O'].tau_beta

AttributeError: 'liN' object has no attribute 'tau_beta'

Note the error! This is due to the fact that the network hasn't been initialized yet. Once the network is initialized, all parameters becomes readily available to both the user and to the SHIP's temporal simulation algorithm. For now, we anticipate that
- the initializaation can be carried out using the `network.init` method (details will be provided in the following tutorial), and
- the information stored during the ```network.add``` or ```network.set_param``` stages can be retrived from two other nested dict properties, init_parameters and init_variables. For instance, a generator function *tau_beta* can be accessed here:

In [40]:
print(net.groups.O.init_parameters.tau_beta)
print("output of the generator function: ", net.groups.O.init_parameters.tau_beta())
net.init()
print("after initialization, the property is stored with the same value: ", net.groups.O.tau_beta)

<function group.set_ki.<locals>.<lambda> at 0x0000020DB5070E50>
output of the generator function:  0.02
after initialization, the property is stored with the same value:  0.02


Here it should be visible that the property `init_parameters.tau_beta` is a generator function, in which SHIP stores the passed arguments; and after network initialization, one can directly access the passed argument in the group property `tau_beta`, as expected.

### Add synapse groups: create connections between neuron groups
The models of the groups above, **poissonN** and **liN**, are defined as ```neurongroup```, i.e. groups that require the defintion of the number of units *N*, and in a conventional network graph representation would correspond to the nodes.

We now want to connect those by way of a synapse model. 
It is possible to do so by importing the desired class, and adding an additional group in very much the same way as done above, with few exceptions. Connections between ```neurongroup``` groups can be carried out by using ```synapsegroup``` groups, which in a conventional network graph representation, correspond to the graph edges. 

`synapsegroup` require two additional keyword arguments with respect to ```neurongroup```: the synapse *source*, and the synapse *target* groups. The number of units, *N*, is instead determined automatically from the connected groups, as SHIP emulates a fully-connected network by default.

We also note that the connection is directed, i.e. signals are carried out exclusively from the source to the target.

We show an example that uses the 2nd order leaky synapse model, here **lS_2o**, also passing as arguments the two temporal constants that specify the model's dynamic behavior, here *tau_alpha1* and *tau_alpha2*.

In [41]:
from SHIP import lS_2o
net.add(lS_2o, 'PO', # class and tag are again mandatory
        source = 'P', target = 'O', # synapse groups eventually require to define source and target
        tau_alpha1 = 8e-3, tau_alpha2 = 4e-3) # group model's parameters.

We note that no value for the synaptic weights is here passed, though a parameter *w* can indeed be passed as an argument. Thus, one may expect to incur in errors or unexpected behavior.

By design, the lS_2o model defines a uniform random distribution between the values of 0 and 1, unless otherwise specified by the user. In fact, every variable and parameter stated in the available models already contain default-values. The user is expected to apply only the necessary changes.

## Syntetic argument notation: passing generator function

This platform has a unique functionality that comes extremely useful during the network building stage. 
SHIP has the option to parse the arguments of the group class as a generator function, which can be called to yield user-determined data. This comes handy for two reasons:
- it allows to re-generate data according to arbitrary distributions every time the network is initialized;
- it also allows to handle the size of the generated tensors, based on the number of the group's components, and/or the number of the parallel batches, with subtle changes of the argument names.

This is the reason why the network needs to be initialized to access to the groups's variables and parameters, and also why the arguments passed during the building stage are stored as generator functions.

### Static argument vs Dynamic argument parsing
We show how SHIP treats the arguments in practice. Let's assume that the user wants to define a randomly-generated set of values for the threshold membrane potential *thr*, along the indices of a LIF group model. To do so, we first import the **lifN** class, an available simple model for LIF neurons. To generate the values, we can use the in-built `torch.rand` function. Then, to set the values of *thr*, the user can go two different ways:

- externally define such values and pass those as a static argument. To do so, one needs to be consistent and provide a set of values (tensor) having the same size as the number of components within the group. This approach fixes both quantities (*N* and *thr*), as a change in one would oblige the user to also change the other (if not - errors ensue).

- the user can alternatively provide merely the function, and tell the platform what the shape of the expected output is. SHIP data parsing that can treat all arguments as generator function, and relate the size of the tensor (here to be set in *thr*) as a function of the number of components *N*. Doing so, one can rapidly adapt the network size and the parameters in the model at the same time. 

See below both examples: 

In [42]:
from SHIP import lifN # importing the lifN class from SHIP

from torch import rand # importing the desired (uniformly-distributed, from 0 to 1) 
                       # random function, already available in pytorch

# static argument method
net.add(lifN,'N_static',N = 5, 
        thr = rand(5))

# dynamic data generation (note the change of the thr argument name, here adding an underscore after it)
net.add(lifN,'N_dynamic',N = 5,
        thr_ = rand)

The static method (top example) defines a set of values once, during the declaration phase. Repeated network initializations would not change such values.

Instead, the dynamic method (bottom example) adds one underscore AFTER the *thr* key, and provide exclusively the torch randfunction. Doing so, the platform acknowledges that the user desires to use the number of components, *N*, as the argument of the generator function (rand).

Below, we see confirmation of the change of the values in *thr* after repeated network initializations. Note the different values in the case of the dynamic method, which confirms that the `init` method generates (through the rand function) new values each time.

In [43]:
print ('first call.')
net.init()
print('static method: theshold values = ',net.groups.N_static.thr)
print('dynamic method: theshold values = ',net.groups.N_dynamic.thr)

print ('second call.')
net.init()
print('static method: theshold values (same as before) = ',net.groups.N_static.thr)
print('dynamic method: theshold values (now different)= ',net.groups.N_dynamic.thr)

first call.
static method: theshold values =  tensor([0.6775, 0.7089, 0.2674, 0.7646, 0.7327])
dynamic method: theshold values =  tensor([0.2349, 0.1658, 0.4397, 0.5792, 0.8435])
second call.
static method: theshold values (same as before) =  tensor([0.6775, 0.7089, 0.2674, 0.7646, 0.7327])
dynamic method: theshold values (now different)=  tensor([0.6978, 0.6470, 0.2042, 0.8737, 0.1829])



### Interpreting the passed arguments: functions and scalars

As shown above, one can pass functions in place of scalars, either built-in or user-determined (`lambda` functions are also ok).
We note however that SHIPS treats the passed argument as a generator function only when the underscore argument is provided *(we must assume that, for any reason, the user would want to provide a non-generator function as an argument, and thus we had to compromise between flexibility of use and a synthetic notation; this feature might change in future versions)*

However, one can just provide a scalar value, and ask SHIP to repeat the value along a tensor dimensions. This is done by passing a scalar value as an argument, and using the underscore notation at the same time. See an example below, where we modify the *thr* value of the example above so to become a tensor of size *N*, and the counterexample using no-underscore notation for the static method:

In [44]:
## static case
net.set_params('N_static', thr = 1.)
print(net.groups.N_static.init_parameters.thr)
print("static case, output of the generator function: ", net.groups.N_static.init_parameters.thr())

# dynamic case
net.set_params('N_dynamic', thr_ = 1.)
print(net.groups.N_dynamic.init_parameters.thr)
print("dynamic case, output of the generator function: ", net.groups.N_dynamic.init_parameters.thr())

# see values after initialization
net.init()
print('static method, threshold values from scalar: thr = ',net.groups.N_static.thr)
print('dynamic method, threshold values from scalar: thr = ',net.groups.N_dynamic.thr)

<function group.set_ki.<locals>.<lambda> at 0x0000020DAD003670>
static case, output of the generator function:  1.0
<function group.set_ki.<locals>.<lambda> at 0x0000020DB5054AF0>
dynamic case, output of the generator function:  tensor([1., 1., 1., 1., 1.])
static method, threshold values from scalar: thr =  1.0
dynamic method, threshold values from scalar: thr =  tensor([1., 1., 1., 1., 1.])


We provided a scalar value. The effect is the following:
- The static method returns a scalar value. Also note no conversion to the tensor datatype is enforced (this is an arbitrary choice, as the user might not want to use the tensor datatype)
- The dynamic method returns a tensor of size *N* = 5 for `net.groups.N_dynamic.thr`, as it has been generated dynamically as if the end user had passed a torch.ones function.  Note that the datatype of the tensor here matches the one of the provided value. Careful! if one does not put the dot after the digit, the assumed datatype is long.


### Size of the dynamically-generated tensor

As shown above, placing one underscore after the argument name tells the 

One can use two underscores in place of one, in case one needs a range of values determined as a function of the number of components of the *source* group and *target* group. This comes useful for ```synapsegroup``` groups, in which the models are generally tuned to work with a 2D matrix of variables, of size determined by the number of source and target components.

See below, where we generate a (uniformly-distributed) random set of values, comprised between 0 and 1, for the temporal constant of the synapse group connecting the lif groups above:

In [45]:
net.add(lS_2o,'S_dynamic',
        source = 'P',
        target = 'N_dynamic',
        tau_alpha1__ = rand)

net.init()
print("The size of P (N=%0.0f) and N_dynamic (N=%0.0f) groups "%(net.groups.P.N,net.groups.N_dynamic.N),
      "would determine a [%0.0fx%0.0f] matrix of %0.0f elements"%(net.groups.P.N,net.groups.N_dynamic.N, net.groups.S_dynamic.N)) 
print("The size of tau_alpha1 is: ", net.groups.S_dynamic.tau_alpha1.shape)

The size of P (N=5) and N_dynamic (N=5) groups  would determine a [5x5] matrix of 25 elements
The size of tau_alpha1 is:  torch.Size([5, 5])


Again - note that we did not specify tau_alpha2, so the default value will be assumed. It is up to the user to verify consistency of the values that the models see and use - and more often than not, a minor mistake on the argument assignment can lead to obscure errors during the simulation of the network temporal evolution. Do double check arguments datatypes and shapes!

Another option is to ask SHIP to use the `batch_size` as an argument for the generator function. This becomes quite useful in case that one wants to gather results in parallel when changing only one parameter. One can tell SHIP to do so using one underscore BEFORE the variable name. 

We anticipate that to set the `batch_size`, one option is to pass the *batch_size* argument during the init function. 

See the example below, where we ask SHIP to modify tau_alpha2 according to the `batch_size`, along with the other parameters, for the group **S_dynamic**:

In [46]:
net.set_params('S_dynamic',_tau_alpha2__ = rand)
net.init(batch_size = 3)
print("The size of P (N=%0.0f) and N_dynamic (N=%0.0f) groups"%(net.groups.P.N,net.groups.N_dynamic.N),
      "would determine a [%0.0fx%0.0f] matrix of %0.0f elements"%(net.groups.P.N,net.groups.N_dynamic.N, net.groups.S_dynamic.N)) 
print("However, the size of tau_alpha2 is now also dependent on the batch_size = %0.0f : "%net.batch_size, net.groups.S_dynamic.tau_alpha2.shape)

The size of P (N=5) and N_dynamic (N=5) groups would determine a [5x5] matrix of 25 elements
However, the size of tau_alpha2 is now also dependent on the batch_size = 3 :  torch.Size([3, 5, 5])


To summarize:
- one can pass any type of argument during the initialization function
- in full generality, numeric values are stored as generator functions, that would generate scalar or tensor values of the same value and datatype as the ones provided by the user
- generator functions can be instructed to use as arguments the batch_size (*\_argname*), the number of components (*argname\_*), or the combined number of components of source and target groups (*argname\__*), by opportunely placing underscores before or after the argument name.
- batch_size and number of component dependencies can be combined

Of course, one need to avoid to write a SHIP model that contains argument names already carrying underscores at the beginning or end of the name.  If that is the case, one needs to modify the model to remove them, otherwise errors will occur.